In [1]:

import sys, os, time, glob
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import tifffile
import seaborn as sns
from nd2reader import ND2Reader
import napari
from napari.utils.colormaps import label_colormap
import itertools
from skimage.filters import gaussian, median, threshold_otsu
from skimage.morphology import remove_small_objects, disk, dilation, erosion, skeletonize, skeletonize_3d, ball 
#from skimage import img_as_ubyte
from skimage.measure import label, regionprops

from scipy.ndimage import binary_fill_holes
from scipy.ndimage import distance_transform_edt, gaussian_filter, binary_dilation, binary_erosion, binary_closing
from scipy.signal import fftconvolve
from scipy.ndimage import maximum_filter, minimum_filter, convolve


# from cellpose import core, utils, io, models, metrics
#sys.path.insert(0,'/home/jmamede/scripts/mamedelab_scripts/notebooks/libraries/')
#from deco_libraries import update_progress,Nd2meta2OME_types

os.environ['LOG_LEVEL'] = 'ERROR'


#$$
def remove_BKG(Image, median_filter_radius=2, min_object_size=500,
               mulitply_threshold=1, title=None, fill_holes=False, plot=False):
    
    sigma = np.array(Image.shape) * 0.25 
    blurred = gaussian(Image, sigma=sigma, preserve_range=True)
    background_subtracted = Image - blurred
    background_subtracted[background_subtracted < 0] = 0
    """
    sigma = np.array(Image.shape) * 0.01
    blurred = gaussian(background_subtracted, sigma=sigma, preserve_range=True)
    background_subtracted = background_subtracted - blurred*0.75
    background_subtracted[background_subtracted < 0] = 0
    """
    # Median filtering
    if Image.ndim == 3:
        median_filtered = np.zeros_like(background_subtracted)
        for z in range(background_subtracted.shape[0]):
            median_filtered[z] = median(background_subtracted[z], disk(median_filter_radius))
    else:
        median_filtered = median(background_subtracted, disk(median_filter_radius))

    # Thresholding
    threshold = threshold_otsu(median_filtered) * mulitply_threshold
    binary_mask = median_filtered > threshold

    # Clean mask
    clean_mask = remove_small_objects(binary_mask, min_size=min_object_size)
    if fill_holes:
        clean_mask = binary_fill_holes(clean_mask)

    median_filtered[clean_mask == 0] = 0

    # Visualization
    if plot:
        if Image.ndim == 3:
            viewer = get_viewer()
            viewer.add_image(Image, name='Original Image', colormap='gray', blending='additive')
            viewer.add_image(background_subtracted, name='background_subtracted Image', colormap='cyan', blending='additive')
            viewer.add_image(median_filtered, name='Median Filtered', colormap='magenta', blending='additive')
            viewer.add_labels(clean_mask.astype(int), name='Clean Mask', blending='additive')
            viewer.dims.ndisplay = 3
        else:
            plt.figure(figsize=(10, 4))
            plt.subplot(1, 3, 1)
            plt.imshow(Image, cmap='gray')
            plt.title("Original Image")
            plt.axis('off')
            plt.subplot(1, 3, 2)
            plt.imshow(median_filtered, cmap='magma')
            plt.title("Median Filtered")
            plt.axis('off')
            plt.subplot(1, 3, 3)
            plt.imshow(clean_mask, cmap='gray')
            plt.title("Clean Mask")
            plt.axis('off')
            if title:
                plt.suptitle(title, fontsize=16)
            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.show()

    return median_filtered, clean_mask
#$$
def connect_skeleton_rays(
    labeled_skeleton,
    min_size=500,
    k_neighbors=5,
    dilate_radius=None,
    fill_holes=True
):
    """
    Connect skeleton pixels within each object using ray tracing.

    Parameters
    ----------
    labeled_skeleton : ndarray
        Labeled skeleton image (2D or 3D).
    min_size : int
        Minimum object size to keep.
    k_neighbors : int
        Number of nearest neighbors to connect for each pixel.
    dilate_radius : int or None
        Optional: radius for binary dilation after connection.
    fill_holes : bool
        If True, fill interior holes after ray connections.

    Returns
    -------
    connected_skeleton : ndarray
        Labeled image with connected regions for each object.
    """

    # Filter small objects
    labeled_skeleton = label(labeled_skeleton > 0)
    labeled_skeleton = remove_small_objects(labeled_skeleton, min_size=min_size)
    labeled_skeleton = label(labeled_skeleton > 0)

    connected_skeleton = np.zeros_like(labeled_skeleton)

    for obj_id in range(1, labeled_skeleton.max() + 1):
        coords = np.argwhere(labeled_skeleton == obj_id)

        if len(coords) == 0:
            continue

        # Start with existing pixels
        connected_skeleton[labeled_skeleton == obj_id] = obj_id

        # Use KDTree for fast neighbor search
        tree = cKDTree(coords)
        _, neighbors = tree.query(coords, k=min(k_neighbors + 1, len(coords)))

        for i, p1 in enumerate(coords):
            for j in neighbors[i][1:]:  # skip self (first neighbor)
                p2 = coords[j]
                rr = line_nd(p1, p2)
                connected_skeleton[tuple(rr)] = obj_id

    # Optional: dilation to thicken lines
    if dilate_radius is not None:
        from skimage.morphology import ball, disk, binary_dilation
        selem = ball(dilate_radius) if connected_skeleton.ndim == 3 else disk(dilate_radius)
        for obj_id in range(1, connected_skeleton.max() + 1):
            mask = connected_skeleton == obj_id
            mask = binary_dilation(mask, selem)
            connected_skeleton[mask] = obj_id

    # Optional: fill interior holes
    if fill_holes:
        from scipy.ndimage import binary_fill_holes
        for obj_id in range(1, connected_skeleton.max() + 1):
            mask = connected_skeleton == obj_id
            mask = binary_fill_holes(mask)
            connected_skeleton[mask] = obj_id

    return connected_skeleton
#$$
def fill_holes_labels_per_slice(label_stack):
    """
    Fill holes in a 3D labeled stack, treating each Z-slice independently
    while preserving label IDs.

    Parameters
    ----------
    label_stack : ndarray
        3D labeled array (Z, Y, X)

    Returns
    -------
    filled_stack : ndarray
        3D labeled array with internal holes filled per label in each slice
    """
    filled_stack = np.zeros_like(label_stack)

    for z in range(label_stack.shape[0]):
        slice_labels = label_stack[z]
        filled_slice = np.zeros_like(slice_labels)

        for label_id in np.unique(slice_labels):
            if label_id == 0:
                continue
            region_mask = slice_labels == label_id
            filled_region = binary_fill_holes(region_mask)
            filled_slice[filled_region] = label_id

        filled_stack[z] = filled_slice

    return filled_stack
#$$
def dilate_labels(skeleton_cell_recon, Accepted_cells, size=1, iterations=50):
    """
    Fill in skeleton labels using max filter while preserving original labels.
    
    Parameters
    ----------
    skeleton_cell_recon : ndarray
        Labeled skeleton array.
    Accepted_cells : ndarray
        Mask of accepted cells.
    size : int
        Maximum filter size.
    iterations : int
        Max number of iterations.

    Returns
    -------
    skeleton_cell_filled : ndarray
        Label-filled skeleton inside Accepted_cells mask.
    """
    skeleton_cell_filled = skeleton_cell_recon.copy()
    previous_sum = 0
    ix = 0

    while (skeleton_cell_filled > 0).sum() < (Accepted_cells > 0).sum() and \
          previous_sum < (skeleton_cell_filled > 0).sum() and ix < iterations:

        previous_sum = (skeleton_cell_filled > 0).sum()

        # Apply max filter to spread labels to neighbors
        temp = maximum_filter(skeleton_cell_filled, size=size)

        # Keep original filled pixels intact
        temp[skeleton_cell_filled > 0] = skeleton_cell_filled[skeleton_cell_filled > 0]

        # Restrict filling to Accepted_cells mask
        skeleton_cell_filled = temp * Accepted_cells

        ix += 1
        
    spill_over = np.zeros_like(skeleton_cell_filled)
    temp = skeleton_cell_filled.copy()
    temp = maximum_filter(temp, size=1)
    spill_over[(temp>0) & (temp!=skeleton_cell_filled)]=1
    temp = skeleton_cell_filled.astype(np.float32)
    temp = temp - (np.max(temp)+1)
    temp2 = temp.copy()
    temp = maximum_filter(temp, size=1)
    spill_over[(temp>0) & (temp!=temp2)]=1
    skeleton_cell_filled[spill_over>0]=0
    del temp, temp2, spill_over
    return skeleton_cell_filled
#$$
def check_masks_overlap(Mask_1 , Mask_2, plot = False):
    # Step 1: Label individual GFP objects
    labeled = label(Mask_1)
    binary = Mask_2.astype(bool)
    # Step 2: Create an empty mask to store only GFP objects that contain DAPI
    Filtered_GFP_Mask = np.zeros_like(Mask_1, dtype=bool)
    # Step 3: Iterate over GFP objects
    for region in regionprops(labeled):
        coords = tuple(zip(*region.coords))
        # Check if any DAPI pixel exists inside this GFP object
        if binary[coords].any() is False:
            # Keep this object
            #Filtered_GFP_Mask[coords] = True
            labeled[coords] = 0
    # Optional: replace original GFP mask
    if plot is not False:
        
        if Image.ndim == 3:
            viewer = get_viewer()
            # Add your images as layers
            viewer.add_labels(Mask_1, name='Mask_1', blending='additive')
            viewer.add_labels(Mask_2, name='Mask_2', blending='additive')
            viewer.add_labels(labeled, name='labeled', blending='additive')
            # If your data is 3D (Z, Y, X), you can enable 3D rendering
            viewer.dims.ndisplay = 3
        else:
            plt.figure(figsize=(10, 4))
            plt.subplot(1, 3, 1)
            plt.imshow(Mask_1, cmap='cyan')
            plt.title("Mask_1")
            plt.axis('off')
            
            plt.subplot(1, 3, 2)
            plt.imshow(Mask_2, cmap='magenta')
            plt.title("Mask_2")
            plt.axis('off')

            plt.subplot(1, 3, 3)
            plt.imshow(labeled, cmap='yellow')
            plt.title("new labeled")
            plt.axis('off')
            plt.show()
    return labeled
#$$
def fast_tiff_metadata(filepath):
    from tifffile import TiffFile
    with TiffFile(filepath) as tif:
        series = tif.series[0]
        return  tif.asarray(), {
            'shape': series.shape,
            'dtype': series.dtype.name,
            'axes': series.axes,
            'metadata': tif.pages[0].tags  # if you want some basic tag info
        }
#$$
def mexican_hat_kernel(size=7, sigma=1.0):
    ax = np.arange(-size // 2 + 1., size // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    r2 = xx**2 + yy**2
    norm = (r2 - 2*sigma**2) / (sigma**4)
    kernel = norm * np.exp(-r2 / (2*sigma**2))
    return kernel
#$$
def generate_condition_combinations(conditions, max_depth=3, return_df=False):
    """
    Generate sorted combinations of given conditions up to a specified depth,
    already respecting the order of `conditions`.
    
    Parameters:
        conditions (list): List of condition names in the desired order.
        max_depth (int): Maximum number of conditions in a combination.
        return_df (bool): If True, return a pandas DataFrame; otherwise, return a list.
    
    Returns:
        list or DataFrame: Sorted combinations as list or DataFrame.
    """
    all_combinations = [
        "_".join(combo)
        for r in range(1, max_depth + 1)
        for combo in itertools.combinations(conditions, r)
    ]
    
    if return_df:
        return pd.DataFrame(all_combinations, columns=["Condition"])
    else:
        return all_combinations
#$$
def get_elongated_cells_old(label_mask, intensity_image=None, 
                        elongation_threshold=0.95, 
                        major_to_minor_axis_ratio=2, 
                        min_object_size=50):
    """
    Identify elongated objects in a labeled mask (2D or 3D).
    
    Returns:
        elongated_regions (list of regionprops)
        labels (list of int)
        properties (list of dict): contains computed metrics per region
    """
    if ndim == 3:
        MP_image = np.max(label_mask, axis=0) 
    else:    
        MP_image = label_mask

    regions = regionprops(label_mask, intensity_image=intensity_image)
    ndim = label_mask.ndim
    elongated_regions = []
    labels = []

    for r in regions:
        if r.area < min_object_size:
            continue
        addcell = False
        prop_dict = {}
        if ndim == 2:
            ecc = r.eccentricity
            axis_ratio = r.axis_major_length / r.axis_minor_length
            prop_dict.update({
                'eccentricity': ecc,
                'major_axis_length': r.axis_major_length,
                'minor_axis_length': r.axis_minor_length,
            })
            if ecc > elongation_threshold and axis_ratio > major_to_minor_axis_ratio:
                addcell=True
                elongated_regions.append(r)
                labels.append(r.label)
                properties.append(prop_dict)

        elif ndim == 3:
            coords = r.coords
            cov = np.cov(coords.T)
            eigvals = np.linalg.eigvalsh(cov)[::-1]  # descending
            if eigvals[-1] == 0:
                continue
            elongation = np.sqrt(eigvals[0] / eigvals[1])

            

            ecc = np.sqrt(1 - (eigvals[1] / eigvals[0]))
            #print(r.label, elongation, eigvals, ecc)
            prop_dict.update({
                #'elongation': elongation,
                'eccentricity': ecc,
                'major_axis_length': np.sqrt(eigvals[0]) * 4,  # approximate
                'minor_axis_length': np.sqrt(eigvals[1]) * 4,  # approximate
            })
            if ecc > elongation_threshold and elongation > major_to_minor_axis_ratio:
                addcell=True
        else:
            raise ValueError("Only 2D or 3D masks are supported.")
        if addcell:
            prop_dict.update({
                #'elongation': elongation,
                "label": r.label,
                "intensity_mean": r.mean_intensity if intensity_image is not None else np.nan,
                'area': r.area, 
            })

            labels.append(r.label)
            elongated_regions.append(prop_dict)

    return elongated_regions, labels
#$$
def get_viewer():
    global viewer
    try:
        # Check if viewer exists and is still alive
        if 'viewer' in globals() and viewer is not None and viewer.window._qt_window.isVisible():
            return viewer
    except RuntimeError:
        # Old viewer was closed
        pass
    
    # Create a new viewer if none exists or old one is dead
    viewer = napari.Viewer(show=True)
    return viewer
#$$
def split_inside_edge_with_labels(
    Accepted_cells, int_image=None, sigma=4,
    intensity_threshold=0.5, calculate_edge=True, plot=False,
    use_mexican_hat=True, mh_size=14, mh_sigma=3.0
):
    inside_cells = np.zeros_like(Accepted_cells)
    edge = np.zeros_like(Accepted_cells)
    ndim = Accepted_cells.ndim

    processed_int = None
    skeleton = None
    closest_skeleton = None
    skeleton_edge = None
    use_mexican_hat = False
    # ---- Intensity processing ----
    if int_image is not None:
        processed_int = np.maximum(int_image - gaussian_filter(int_image, sigma=3),0)
        processed_int[processed_int < 0] = 0
        if use_mexican_hat:
            kernel = mexican_hat_kernel(size=mh_size, sigma=mh_sigma)
            if ndim == 2:
                processed_int = fftconvolve(processed_int, kernel, mode='same')
            elif ndim == 3:
                processed_int = np.zeros_like(int_image, dtype=float)
                for z in range(int_image.shape[0]):
                    processed_int[z] = fftconvolve(int_image[z], kernel, mode='same')
        processed_int[processed_int < 0] = 0
        # Skeleton per slice
        skeleton =  np.zeros_like(processed_int, dtype=bool)
        for label_id in np.unique(Accepted_cells):
                
                skeleton[Accepted_cells == label_id] =  processed_int[Accepted_cells == label_id] >  processed_int[Accepted_cells == label_id].max()  * intensity_threshold #- processed_int[Accepted_cells == label_id] 


        if ndim == 2:
            skeleton = skeletonize(skeleton)
        else:
            #skeleton = np.zeros_like(processed_int, dtype=bool)
            for z in range(processed_int.shape[0]):
                skeleton[z] = skeletonize(skeleton[z])
        #skeleton = remove_small_objects(skeleton, min_size=20)
        
        skeleton_cell_recon = label(skeleton)
        skeleton_cell_recon = remove_small_objects(skeleton_cell_recon, min_size=500)
        skeleton_cell_recon = label(skeleton_cell_recon > 0)
        skeleton_props = regionprops(skeleton_cell_recon)

        # Example: print object sizes
        sizes = [prop.area for prop in skeleton_props]
        # New filled reconstruction
        skeleton_cell_filled = dilate_labels(skeleton_cell_recon, processed_int>0, iterations=50, size=3)
        Accepted_cells = fill_holes_labels_per_slice(skeleton_cell_filled)
        edge = dilate_labels(skeleton_cell_recon, processed_int>0, iterations=2,size=3)
        temp = np.zeros_like(edge, dtype=bool)
        if ndim == 3:
            for z in range(Accepted_cells.shape[0] ):
                temp[z] = minimum_filter(Accepted_cells[z], size=9)
        else:
            temp = minimum_filter(Accepted_cells, size=9)
        edge[temp>0]=0

        

        
    # ---- Edge detection per cell ----
    if calculate_edge and skeleton is None:
        for label_id in np.unique(Accepted_cells):
            if label_id == 0:
                continue
            mask = Accepted_cells == label_id

            # Edge for this cell
            if ndim == 3:
                cell_edge = np.zeros_like(mask, dtype=bool)
                for z in range(mask.shape[0]):
                    cell_edge[z] = binary_dilation(mask[z], disk(3)) & ~binary_erosion(mask[z], disk(3)) & mask[z]
            else:
                cell_edge = binary_dilation(mask, disk(2)) & ~binary_erosion(mask, disk(2)) & mask

            edge[cell_edge] = label_id


           
            #return   skeleton, cell_edge
            # Find skeleton pixels closest to each edge pixel
            """
            if skeleton is not None:

                if skeleton_edge is None:
                    skeleton_edge = np.zeros_like(skeleton, dtype=bool)
               
                closest_skeleton = np.zeros_like(skeleton, dtype=bool)

                for z in range(cell_edge.shape[0] if ndim == 3 else 1):
                    slice_edge = cell_edge[z] if ndim == 3 else cell_edge
                    slice_skel = skeleton[z] if ndim == 3 else skeleton

                    if np.any(slice_edge) and np.any(slice_skel):
                        dist_map, indices = distance_transform_edt(~slice_skel, return_indices=True)
                        skel_y = indices[0][slice_edge]
                        skel_x = indices[1][slice_edge]
                        if ndim == 3:
                            skeleton_edge[z, skel_y, skel_x] = True
                            closest_skeleton[z, skel_y, skel_x] = True
                        else:
                            skeleton_edge[skel_y, skel_x] = True
                            closest_skeleton[skel_y, skel_x] = True

                #closest_skeleton = (closest_skeleton, ball(1) if ndim == 3 else disk(1))  
                    
                if ndim == 3:
                    for z in range(cell_edge.shape[0] if ndim == 3 else 1):     
                        closest_skeleton[z] = binary_dilation(closest_skeleton[z],  disk(1)) 
                        closest_skeleton[z] = binary_closing(closest_skeleton[z],  disk(1), border_value=0)
                else:
                    closest_skeleton = binary_dilation(closest_skeleton,  disk(1))
                    closest_skeleton = binary_closing(closest_skeleton,  disk(1), border_value=0)
                #closest_skeleton = binary_closing(closest_skeleton, ball(2) if ndim == 3 else disk(2), border_value=0 )  
                skeleton_edge = skeleton_edge | closest_skeleton
            """

    inside_cells = Accepted_cells.copy()
    inside_cells[edge>0] = 0
    

    
    # ---- Plot results ----
    if plot:
        if ndim == 3:
            viewer = get_viewer()

            viewer.add_labels(inside_cells, name='inside_cells' if skeleton is None else "inside_cells skeleton" , blending='additive')
            viewer.add_labels(edge, name='edge' if skeleton is None else "edge skeleton", blending='additive')
            viewer.add_labels(Accepted_cells, name='Accepted_cells' if skeleton is None else "Accepted_cells - skeleton filled", blending='additive')
            if processed_int is not None:
                #viewer.add_labels(temp, name='Temp inside???', blending='additive')
                viewer.add_image(processed_int, name='Processed Intensity', colormap='magma', blending='additive')
            viewer.dims.ndisplay = 3
        else:
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 3, 1)
            plt.imshow(inside_cells, cmap='cyan')
            plt.title("Inside cells"); plt.axis('off')

            plt.subplot(1, 3, 2)
            plt.imshow(edge, cmap='magenta')
            plt.title("Cell edge"); plt.axis('off')

            if closest_skeleton is not None:
                plt.subplot(1, 3, 3)
                plt.imshow(closest_skeleton, cmap='yellow')
                plt.title("Closest Skeleton"); plt.axis('off')
            plt.show()

    return inside_cells, edge, Accepted_cells
#$$
def get_elongated_cells(label_mask, intensity_image=None,
                        elongation_threshold=0.95,
                        major_to_minor_axis_ratio=2,
                        min_object_size=50,
                        use_skeleton=True,
                        plot=False):
    """
    Identify elongated objects in a labeled mask (2D or 3D).
    
    Returns:
        elongated_regions (list of dict)
        labels (list of int)
    """
    from skimage.morphology import skeletonize
    from skimage.graph import MCP_Geometric  # fast geodesic pathfinding on skeletons

    def longest_skeleton_branch_length_fast(skel):
        """Find the longest branch length in a skeleton using geodesic distance."""
        if skel.sum() == 0:
            return 0

        # Count neighbors to find endpoints
        kernel = np.ones((3, 3), dtype=int)
        kernel[1, 1] = 0
        neighbor_count = convolve(skel.astype(int), kernel, mode='constant')
        endpoints = np.argwhere(skel & (neighbor_count == 1))

        if len(endpoints) < 2:
            return skel.sum()  # no branches → length = total skeleton length

        # Prepare MCP for geodesic distance
        cost_array = np.where(skel, 1, np.inf)
        mcp = MCP_Geometric(cost_array)

        max_len = 0
        # Compute distances only from endpoints
        for i, ep in enumerate(endpoints):
            costs, _ = mcp.find_costs([tuple(ep)])
            # Max distance to any other endpoint
            endpoint_costs = costs[tuple(endpoints.T)]
            if np.isfinite(endpoint_costs).any():
                max_len = max(max_len, np.nanmax(endpoint_costs))

        return max_len


    ndim = label_mask.ndim
    elongated_regions = []
    labels = []
    
    regions = regionprops(label_mask, intensity_image=intensity_image)
    skeletons = np.zeros(label_mask.shape[-2:],dtype=np.uint16)


    for r in regions:
        if r.area < min_object_size:
            continue

        prop_dict = {
            "label": r.label,
            "area": r.area,
            "intensity_mean": r.mean_intensity if intensity_image is not None else np.nan
        }

        if ndim == 2:
            mask = (label_mask == r.label)
            if use_skeleton:
                skel = skeletonize(mask)
                skeletons[skel>0]=r.label
                skel_length = longest_skeleton_branch_length_fast(skel)
                elongation = (skel_length ** 2) / r.area
            else:
                ecc = r.eccentricity
                axis_ratio = r.axis_major_length / max(r.axis_minor_length, 1e-6)
                elongation = axis_ratio  # fallback

        elif ndim == 3:
            mask = (label_mask == r.label)
            if use_skeleton:
                MP_image = np.max(mask, axis=0)
                skel = skeletonize(MP_image)
                #skel = skeletonize_3d(mask)
                skeletons[skel>0]=r.label
                skel_length = longest_skeleton_branch_length_fast(skel)
                elongation = (skel_length ** 2) / np.sum(MP_image)
            else:
                coords = r.coords
                cov = np.cov(coords.T)
                eigvals = np.linalg.eigvalsh(cov)[::-1]
                if eigvals[-1] == 0:
                    continue
                elongation = np.sqrt(eigvals[0] / eigvals[1])

        # Store results if elongated enough
        if elongation > major_to_minor_axis_ratio:
            prop_dict["elongation"] = elongation
            labels.append(r.label)
            elongated_regions.append(prop_dict)
    if plot:
        if ndim == 3:
            viewer = get_viewer()
            viewer.add_labels(skeletons, name='SkeletonMP'  , blending='additive')
            viewer.add_labels(label_mask, name='Label Mask', blending='additive')
            viewer.dims.ndisplay = 3
    return elongated_regions, labels
#$$

c:\Users\jszczerkowski\AppData\Local\anaconda3\envs\tf_env\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
#!from bioio import BioImage
print(sys.executable)

c:\Users\jszczerkowski\AppData\Local\anaconda3\envs\tf_env\python.exe


In [26]:


#FolderOI = r"D:\Xana\2025.08.20_gp CM Male_pPKA" #  ran
#FolderOI = r"D:\Xana\2025.09.03_gp CM Male_pPKA"
FolderOI = r"D:\Xana\2025.09.08_gp CM Female_pPKA"

filelist = glob.glob(FolderOI+os.path.sep+"*.tif")
# Group filelist by the part before _C0/_C1
paired_files = defaultdict(dict)
for f in filelist:
    base = f.rsplit('_C', 1)[0]  # everything before _C0 or _C1
    channel = f[-5]  # '0' or '1' from '_C0' or '_C1'
    paired_files[base][channel] = f

# Convert to list of tuples (C0, C1)
pairs = [(v['0'], v['1']) for v in paired_files.values() if '0' in v and '1' in v]

# Print results
for c0, c1 in pairs:
    print(c0, "<--->", c1)




D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 10_XY1757341712_Z00_T0_C0.tif <---> D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 10_XY1757341712_Z00_T0_C1.tif
D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 11_XY1757341765_Z00_T0_C0.tif <---> D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 11_XY1757341765_Z00_T0_C1.tif
D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 12_XY1757341813_Z00_T0_C0.tif <---> D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 12_XY1757341813_Z00_T0_C1.tif
D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 13_XY1757341862_Z00_T0_C0.tif <---> D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 13_XY1757341862_Z00_T0_C1.tif
D:\Xana\2025.09.08_gp CM Female_pPKA\2025.09.08_gp CM Female_pPKA647Dapi_2000 - 14_XY1757341917_

In [27]:

if "viewer" in globals() and globals()["viewer"] is not None:
    try:
        globals()["viewer"].close()
    except RuntimeError:
        pass  # viewer already deleted
    finally:
        globals()["viewer"] = None


%matplotlib qt
#plt.close('all')  

min_object_size = 30000  # Adjust based on your noise size
subtract =2000
Conditions = ['DMSO',  'CGS', '2000', 'ISO', 'IBMX+Fsk', 'H89']

channel_names = ['Dapi', 'pPKA']
Dapi=0
proteinsOI = [1]
Over_threshold = []

elongation_threshold = 0.95
major_to_minor_axis_ratio = 3.0
maxproject =False
calculate_edge =True


plot=False
recalculate=True

Results=[]
for idx, file in enumerate(pairs):
    if recalculate is False: break
    #if idx <10: continue
    print(f"Processing file {idx+1}/{len(pairs)}: {os.path.basename(file[0])}")
    Dapi_Image, Dapi_Metadata =fast_tiff_metadata(file[Dapi])
    axes=Dapi_Metadata['axes'].lower().replace('i','z')
    #image2 = tifffile.imread(file[1])
    if maxproject: 
        Dapi_Image = np.max(Dapi_Image, axis=0) 
        axes = axes.replace('z','')  # remove z from axes if maxproject
    
    Dapi_Image, Dapi_Mask = remove_BKG(Dapi_Image, median_filter_radius=2, min_object_size=250, mulitply_threshold = 0.8, title=f"Dapi: Ch{Dapi} - {channel_names[Dapi]}",
                                fill_holes=True, plot=False)
    
    for chan in proteinsOI:
        Image, Metadata =fast_tiff_metadata(file[chan])
        if maxproject: Image = np.max(Image, axis=0) 
        Image_filtered, Mask = remove_BKG(Image, median_filter_radius=2, min_object_size=250, mulitply_threshold = 0.5, title=f"ProteinOI: Ch{chan} - {channel_names[chan]}",
                                fill_holes=True, plot=False)
        
        Accepted_cells = check_masks_overlap(Mask , Dapi_Mask, plot = False)

        inside_cells, edge , Accepted_cells  = split_inside_edge_with_labels(Accepted_cells, Image_filtered,   intensity_threshold=0.2,  # fraction of max intensity
                        use_mexican_hat=True, mh_size=9, mh_sigma=2.0,
                            calculate_edge=calculate_edge, plot=plot)

        regions = regionprops(Accepted_cells, intensity_image=Image)

        elongated_cells, labels,  = get_elongated_cells(
                Accepted_cells,
                intensity_image=Image,
                major_to_minor_axis_ratio=5,
                min_object_size=min_object_size,
                plot=plot
            )

        for cell in elongated_cells:
            #print(f"Label {cell['label']}, Area: {cell['area']}, Elongation: {cell.get('elongation', np.nan):.2f}, Intensity Mean: {cell['intensity_mean']:.2f}")
            Condition_found = [con for con in Conditions if con in file[chan]]
            Condition_found = ('_').join(Condition_found)
            cell.update(dict(file=os.path.basename(file[chan]), filepath=file[chan],
                       Parent_folder=os.path.split(os.path.split(file[chan])[0])[1],
                       Parent_path=os.path.split(os.path.split(file[chan])[0])[0],
                       Condition=Condition_found))

            cell['mean_intensity_filtered'] = np.mean(Image_filtered[Accepted_cells==cell['label']])
            if calculate_edge:
                cell_ID = cell['label']
                edge_mean = np.mean(Image[edge==cell_ID])
                cyto_mean = np.mean(Image[inside_cells==cell_ID])
                cell['edge_mean_intensity'] = edge_mean
                cell['cyto_mean_intensity'] = cyto_mean
                #cell['edge_to_cyto_ratio'] = cyto_mean / edge_mean if cyto_mean != 0 else np.nan
            Results.append(cell)
            #print(cell)
    if plot: break

Parent, FolderName = os.path.split(FolderOI)
if recalculate: 
    #fsdfdsf
    Results = pd.DataFrame(Results) 
    Results.to_csv(os.path.join(Parent,f"Results for {FolderName}.csv"), index=False)  
else:
    Results = pd.read_csv(os.path.join(Parent,f"Results for {FolderName}.csv")) 


Processing file 1/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 10_XY1757341712_Z00_T0_C0.tif
Processing file 2/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 11_XY1757341765_Z00_T0_C0.tif
Processing file 3/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 12_XY1757341813_Z00_T0_C0.tif
Processing file 4/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 13_XY1757341862_Z00_T0_C0.tif
Processing file 5/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 14_XY1757341917_Z00_T0_C0.tif
Processing file 6/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 15_XY1757341966_Z00_T0_C0.tif
Processing file 7/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 16_XY1757342030_Z00_T0_C0.tif
Processing file 8/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 17_XY1757342073_Z00_T0_C0.tif
Processing file 9/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 18_XY1757342119_Z00_T0_C0.tif
Processing file 10/121: 2025.09.08_gp CM Female_pPKA647Dapi_2000 - 19_XY1757342168_Z00_T0_C0.tif
Processing file 11/121: 2025.09.08_gp C

In [30]:
print(Parent,FolderName)


D:\Xana 2025.09.08_gp CM Female_pPKA


In [29]:
# Example usage

condition_order = generate_condition_combinations(Conditions, max_depth=3, return_df=False)
condition_order = [con for con in condition_order if con in Results["Condition"].unique()]

 

In [31]:
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
from itertools import combinations

control = "" #"DMSO"
plot_orders = ['DMSO', '2000', 'CGS',  'CGS+2000', 'ISO', 'IBMX+Fsk', 'H89', 'ISO+H89', 'IBMX+Fsk+H89', 'CGS+2000+H89']

plt.close('all')  
df=Results.copy()
df['Condition'] = df['Condition'].str.replace('_', '+')
df["Condition"] = pd.Categorical(df["Condition"], categories=plot_orders, ordered=True)
df = df.sort_values("Condition")
df['Condition'] = df['Condition'].astype(str)
df['Condition'] = df['Condition'].apply(lambda x: x.replace('_', '+') if pd.notnull(x) else x)
df = df[~df['Condition'].str.contains('H89')]


df["edge / cyto"] = df['edge_mean_intensity'] / df['cyto_mean_intensity']
df["cyto / edge"] = df['cyto_mean_intensity'] / df['edge_mean_intensity']



plot_order = [ i for i in plot_orders if i in df['Condition'].unique()]
if control !="":
    pairs = [(control, cond) for cond in plot_order if cond != control]
else:
    pairs = list(combinations(plot_order, 2))  # all vs all comparisons




# Pick colors for each condition in order
palette_colors = sns.color_palette("pastel", len(plot_orders))

# Map each condition to its color
condition_colors = dict(zip(plot_orders, palette_colors))




df["edge / cyto"] = df['edge_mean_intensity'] / df['cyto_mean_intensity']
df["cyto / edge"] = df['cyto_mean_intensity'] / df['edge_mean_intensity']

# Assume plot_order is your sorted list of conditions

def barplot_with_stats(data, y, title):
    plt.figure(figsize=(8, 6))
    
    # Barplot without hue to avoid side-by-side dodging
    ax = sns.barplot(
        data=data,
        x="Condition",
        y=y,
        ci='sd',
        edgecolor="black",
        palette=condition_colors,
        order=plot_order
    )
    
    # Stripplot overlay
    sns.stripplot(
        data=data,
        x="Condition",
        y=y,
        palette=condition_colors,
        alpha=0.6,
        jitter=True,
        order=plot_order
    )
    
    # Statistical annotations vs DMSO
    annotator = Annotator(ax, pairs, data=data, x="Condition", y=y, order=plot_order)
    annotator.configure(
        test='Mann-Whitney', 
        text_format='star', 
        loc='inside',            # put lines inside axis
        hide_non_significant=True,  # hide ns labels
        verbose=0
    )
    annotator.apply_and_annotate()
    
    plt.ylabel(y.replace("_", " ").title())
    plt.xlabel("Conditions")
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
 # Define parent folder and subfolder
    #Parent = r"D:\Xana\Jamie"  # The folder where the file will be saved
    #FolderName = "2025.09.03_gp CM Male_pPKA"  # Example dynamic part
    #title = "title"  # Your plot title or additional descriptor

    # Ensure parent folder exists
    os.makedirs(Parent, exist_ok=True)

    # Build save path
    save_path = os.path.join(Parent,"Jamie", f"{FolderName}_{title}.png")
    print(save_path)
    # Save the plot
    plt.savefig(save_path, dpi=1000)


    
    plt.show()




# Run plots
barplot_with_stats(df, "cyto / edge", "Cyto vs Edge - Mean Intensity by Condition")
barplot_with_stats(df, "edge / cyto", "Edge vs Cyto - Mean Intensity by Condition")
barplot_with_stats(df, "mean_intensity_filtered", "Mean Intensity (-BKG) by Condition")
barplot_with_stats(df, "intensity_mean", "Mean Intensity (Org) by Condition")


D:\Xana\Jamie\2025.09.08_gp CM Female_pPKA_Cyto vs Edge - Mean Intensity by Condition.png
D:\Xana\Jamie\2025.09.08_gp CM Female_pPKA_Edge vs Cyto - Mean Intensity by Condition.png
D:\Xana\Jamie\2025.09.08_gp CM Female_pPKA_Mean Intensity (-BKG) by Condition.png
D:\Xana\Jamie\2025.09.08_gp CM Female_pPKA_Mean Intensity (Org) by Condition.png


In [ ]:
df['Condition'].unique()

In [ ]:
order_map

In [ ]:
cell_ID=6
cell = [idx for idx,r in enumerate(elongated_cells) if r.label == cell_ID][0]
print("   ---- Cell:", elongated_cells[cell].label)
print("eccentricity: \t\t\t",elongated_cells[cell].eccentricity)
print("equivalent_diameter_area: \t",elongated_cells[cell].equivalent_diameter_area)
print("axis_major_length: \t\t\t",elongated_cells[cell].axis_major_length)
print("axis_minor_length: \t\t",elongated_cells[cell].axis_minor_length)
print("axis_length_ratio: \t\t",elongated_cells[cell].axis_major_length/elongated_cells[cell].axis_minor_length)
print("area: \t\t\t\t",elongated_cells[cell].area)



cell_ID=3
cell = [idx for idx,r in enumerate(elongated_cells) if r.label == cell_ID][0]
print("   ---- Cell:", elongated_cells[cell].label)
print("eccentricity: \t\t\t",elongated_cells[cell].eccentricity)
print("equivalent_diameter_area: \t",elongated_cells[cell].equivalent_diameter_area)
print("axis_major_length: \t\t\t",elongated_cells[cell].axis_major_length)
print("axis_minor_length: \t\t",elongated_cells[cell].axis_minor_length)
print("axis_length_ratio: \t\t",elongated_cells[cell].axis_major_length/elongated_cells[cell].axis_minor_length)
print("area: \t\t\t\t",elongated_cells[cell].area)

row